## Run the blocks below only for the first time
### Install dependencies

In [ ]:
%%bash
sh env.sh

### Download pretrained models (may take a minute)

In [2]:
import yaml
import sys
import requests
import os
import re
import tarfile
import shutil
# for zip file
import zipfile


# url = "https://onedrive.live.com/?cid=5435770760f02d2f&id=5435770760F02D2F%211152&authkey=!AD6jcLtyet8Y3o4"
    
# r = requests.get(url, allow_redirects=True)

# open('cpvton-plus-checkpoints.zip', 'wb').write(r.content)

def download(url, filename, cookies=None):
    with open(filename, 'wb') as f:
        response = requests.get(url, stream=True, cookies=cookies)
        total = response.headers.get('content-length')

        if total is None:
            f.write(response.content)
        else:
            downloaded = 0
            total = int(total)
            for data in response.iter_content(chunk_size=max(int(total/1000), 1024*1024)):
                downloaded += len(data)
                f.write(data)
                completed = int(50*downloaded/total)
                sys.stdout.write('\r[{}{}]'.format(
                    '█' * completed, '.' * (50-completed)))
                sys.stdout.flush()
    sys.stdout.write('\n')


In [2]:
drive_request = requests.get(
    'https://drive.google.com/uc?export=download&confirm=CONFIRM&id=199dwgFGlumJ95MAGNbLnYtbVKA2w6kdc')
confirm_page = drive_request.text
confirmation_code = re.findall('confirm=(.{4})', confirm_page)[0]

print('[*] Downloading data...')
download('https://drive.google.com/uc?export=download&confirm=CONFIRM&id=199dwgFGlumJ95MAGNbLnYtbVKA2w6kdc'.replace(
    'CONFIRM', confirmation_code), 'cpvton-plus-checkpoints.zip', cookies=drive_request.cookies)
print('[^] Download complete')

[*] Downloading data...

[^] Download complete


### Excuate the block below once you see a 'cpvton-plus-checkpoints.zip' in the storage directory
#### Extract the checkpoint to 'storage/checkpoints-cpvton-plus' directory and delete the zip file

In [ ]:
if not os.path.exists('checkpoints-cpvton-plus'):
    os.makedirs('checkpoints-cpvton-plus')
checkpoint = zipfile.ZipFile("cpvton-plus-checkpoints.zip")
checkpoint.extractall(path='checkpoints-cpvton-plus')
shutil.move('checkpoints-cpvton-plus/checkpoints/GMM/', 'checkpoints-cpvton-plus/GMM/')
shutil.move('checkpoints-cpvton-plus/checkpoints/TOM/', 'checkpoints-cpvton-plus/TOM/')
os.rmdir('checkpoints-cpvton-plus/checkpoints')
os.remove('cpvton-plus-checkpoints.zip')

In [ ]:
os.rmdir('checkpoints-cpvton-plus/checkpoints')

### Download the whole VITON datasets and relocate the data (may take 20 minutes or so)

In [ ]:
if not os.path.exists('data'):
    os.makedirs('data')

In [ ]:

drive_request = requests.get(
    'https://drive.google.com/uc?export=download&confirm=CONFIRM&id=1MxCUvKxejnwWnoZ-KoCyMCXo3TLhRuTo')
confirm_page = drive_request.text
confirmation_code = re.findall('confirm=(.{4})', confirm_page)[0]

print('[*] Downloading data...')
download('https://drive.google.com/uc?export=download&confirm=CONFIRM&id=1MxCUvKxejnwWnoZ-KoCyMCXo3TLhRuTo'.replace(
    'CONFIRM', confirmation_code), 'data/viton_resize.tar.gz', cookies=drive_request.cookies)

tarfile.open("data/viton_resize.tar.gz").extractall(path='data/')

shutil.move('data/viton_resize/test/', 'data/test/')
shutil.move('data/viton_resize/train/', 'data/train/')

os.rmdir('data/viton_resize/')
os.remove('data/viton_resize.tar.gz')

## Run the blocks above only for the first time
### Run blocks below for test stage
#### Change directory to root 
##### Run blocks below once you see train and test folders in 'storage/data' directory

In [1]:
import os
os.chdir('../notebooks')
if (os. getcwd() == '/notebooks'):
    print('Now working in the right location')
else:
    print('ERROE! Current directory is:', os. getcwd())
        

Now working in the right location


### Build GMM results to wrap the clothes based on test models' poses (I changed the python scripts directly so below commends run by bash)
### Each lines may take a long time to run
#### You can check the results after the procedure finished

### Preprocessing Stage 
#### Generate stage one sagementation results with neck features

In [ ]:
%%bash
python dataset_neck_skin_correction.py

In [ ]:
%%bash
python body_binary_masking.py

### CIHP_PGN pretrained model

In [ ]:
drive_request = requests.get(
    'https://drive.google.com/uc?export=download&confirm=CONFIRM&id=1Mqpse5Gen4V4403wFEpv3w3JAsWw2uhk')
confirm_page = drive_request.text
confirmation_code = re.findall('confirm=(.{4})', confirm_page)[0]

print('[*] Downloading data...')
download('https://drive.google.com/uc?export=download&confirm=CONFIRM&id=1Mqpse5Gen4V4403wFEpv3w3JAsWw2uhk'.replace(
    'CONFIRM', confirmation_code), 'CIHP_PGN-checkpoints.zip', cookies=drive_request.cookies)
print('[^] Download complete')
if not os.path.exists('checkpoints-CIHP_PGN'):
    os.makedirs('checkpoints-CIHP_PGN')
checkpoint = zipfile.ZipFile("CIHP_PGN-checkpoints.zip")
checkpoint.extractall(path='checkpoints-CIHP_PGN')

os.remove('CIHP_PGN-checkpoints.zip')

### GMM results

In [ ]:
%%bash
python test.py --name GMM --dataroot storage/data --stage GMM --workers 4 --datamode test --data_list test_pairs.txt --checkpoint storage/checkpoints-cpvton-plus/checkpoints/GMM/gmm_final.pth


In [ ]:
%%bash
cp -r storage/result-cpvton-plus/GMM/test/warp-cloth storage/data/test
cp -r storage/result-cpvton-plus/GMM/test/warp-mask storage/data/test



In [ ]:
%%bash
python test.py --name TOM --dataroot storage/data --stage TOM --workers 4 --datamode test --data_list test_pairs.txt --checkpoint storage/checkpoints-cpvton-plus/TOM/tom_final.pth

In [ ]:
%%bash
mv -r result storage/result

 ### The results for cp-vton-plus now can be checked in 'storage/result/TOM/test/try-on'

### Training Stage

In [ ]:
%%bash
python train.py --name GMM --dataroot storage/data --stage GMM --workers 4 --decay_step 35000 --keep_step 35000 --data_list train_pairs.txt --checkpoint_dir storage/checkpoints-cpvton-plus/ --save_count 5000 --shuffle --tensorboard_dir storage/tensorboard-CPVTON-plus

In [ ]:
%%bash
python test.py --name GMM --dataroot storage/data --stage GMM --workers 4 --datamode train --data_list train_pairs.txt --checkpoint storage/checkpoints-cpvton-plus/GMM/gmm_raw.pth --result_dir storage/result-cpvton-plus 

In [ ]:
%%bash
cp -r storage/result-cpvton-plus/GMM/train/warp-cloth storage/data/train
cp -r storage/result-cpvton-plus/GMM/train/warp-mask storage/data/train

In [ ]:
%tensorboard  --logdir= storage/tensorboard-CPVTON-plus

In [ ]:
%load_ext tensorboard

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [ ]:
python train.py --name TOM --dataroot storage/data --stage TOM --workers 4 --decay_step 35000 --keep_step 35000 --data_list train_pairs.txt --checkpoint_dir storage/checkpoints-cpvton-plus/ --save_count 5000 --shuffle --tensorboard_dir storage/tensorboard-CPVTON-plus

In [ ]:
pip install coremltools

In [9]:
# Jen added: Trace model
import torch
import torchvision

In [ ]:
torch_model = torch.load('storage/checkpoints-cpvton-plus/TOM/tom_raw.pth')
torch_model.eval()

In [11]:
# MARK: Please try the tracing method here

example_input = torch.rand(64, 26, 192,256)
traced_model = torch.jit.trace(torch_model,example_input.cuda())

In [12]:
print(torch.__version__)

1.6.0


In [6]:
import os
import zipfile

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))




In [5]:
zipf = zipfile.ZipFile('train-opt.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('storage/data/train-opt', zipf)
zipf.close()

In [ ]:
%%bash
python cloth_binary_masking.py

In [ ]:
python train.py --name GMM --dataroot storage/data --datamode train-opt --stage GMM --workers 4 --decay_step 35000 --keep_step 35000 --data_list train_pairs.txt --checkpoint_dir storage/checkpoints-cpvton-plus/ --save_count 5000 --shuffle --tensorboard_dir storage/tensorboard-CPVTON-plus

In [ ]:
python test.py --name GMM --dataroot storage/data --stage GMM --workers 4 --datamode train-opt --data_list train_pairs.txt --checkpoint storage/checkpoints-cpvton-plus/GMM/gmm_opt.pth --result_dir storage/result-cpvton-plus-opt 

In [ ]:
%%bash
cp -r storage/result-cpvton-plus-opt/GMM/train-opt/warp-cloth storage/data/train-opt
cp -r storage/result-cpvton-plus-opt/GMM/train-opt/warp-mask storage/data/train-opt

In [ ]:
%%bash
python test.py --name GMM --dataroot storage/data --stage GMM --workers 4 --datamode test-end2end --data_list train_pairs.txt --checkpoint storage/checkpoints-cpvton-plus/GMM/gmm_raw.pth --result_dir storage/result-cpvton-plus 

### new mission: TRAIN TOM

In [ ]:
python train.py --name TOM --dataroot storage/data --stage TOM --workers 4 --datamode train-opt --data_list train_pairs.txt --checkpoint_dir storage/checkpoints-cpvton-plus/ --save_count 5000 --shuffle --tensorboard_dir storage/tensorboard-CPVTON-plus

In [3]:
drive_request = requests.get(
    'https://drive.google.com/uc?export=download&confirm=CONFIRM&id=1B-__YtFz9S8zXWiwF4WEoFh_213dHaaC')
confirm_page = drive_request.text
confirmation_code = re.findall('confirm=(.{4})', confirm_page)[0]

print('[*] Downloading data...')
download('https://drive.google.com/uc?export=download&confirm=CONFIRM&id=1B-__YtFz9S8zXWiwF4WEoFh_213dHaaC'.replace(
    'CONFIRM', confirmation_code), 'joint-points-test.zip', cookies=drive_request.cookies)
print('[^] Download complete')




[*] Downloading data...

[^] Download complete


In [ ]:
checkpoint = zipfile.ZipFile("joint-points-test.zip")
checkpoint.extractall(path='storage/data/')

In [ ]:
python segamentation/simple_extractor.py --dataset 'lip' --model-restore storage/checkpoints-humanParsing/exp-schp-201908261155-lip.pth --input-dir storage/data/test-end2end/image --output-dir storage/data/test-end2end/image-parse

In [ ]:
python test.py --name GMM --dataroot storage/data --stage GMM --workers 4 --datamode test-end2end --data_list test_end2end_pairs.txt --checkpoint storage/checkpoints-cpvton-plus/GMM/gmm_opt.pth --result_dir storage/result-cpvton-plus-testend2end 

## Test TOM by end2end dataset

### check working directory

In [35]:
import os, subprocess, sys, datetime, signal, shutil
os.chdir('../notebooks')
if (os. getcwd() == '/notebooks'):
    print('Now working in the right location')
else:
    print('ERROE! Current directory is:', os. getcwd())
        

Now working in the right location


### Clear the test dataset

In [40]:
cmd_clear = "rm -r storage/data/test-end2end"
subprocess.call(cmd_clear, shell=True)

0

### Write cloth image pair to 'storage/data/test_end2end_pair.txt'
#### image and cloth must be exist in storage/data/test/image and storage/data/test/cloth
#### e.g. 000010_0.jpg 000001_1.jpg 

### Initial test files

In [41]:
dataroot = 'storage/data/'
data_list = 'test_end2end_pairs.txt'
cloth_src = 'storage/data/test/cloth'
image_src = 'storage/data/test/image'
cloth_dir = 'storage/data/test-end2end/cloth'
image_dir = 'storage/data/test-end2end/image'
pose_src = 'storage/data/test-opt/pose'
pose_dir = 'storage/data/test-end2end/pose'
im_names = []
c_names = []
if not os.path.exists(cloth_dir):
    os.makedirs(cloth_dir)
if not os.path.exists(image_dir):
    os.makedirs(image_dir)
if not os.path.exists(pose_dir):
    os.makedirs(pose_dir)
with open(os.path.join(dataroot, data_list), 'r') as f:
            for line in f.readlines():
                im_name, c_name = line.strip().split()
                im_names.append(im_name)
                c_names.append(c_name)
def copy_files_dir(files_list, src, dest):
    src_files = os.listdir(src)
    for file_name in src_files:
        if file_name in files_list:
            shutil.copy(os.path.join(src, file_name), dest)
poses = os.listdir(pose_src)
for pose in poses:
    image_name = pose.split('_')[0]+'_0.jpg'
    if image_name in im_names:
        shutil.copy(os.path.join(pose_src, pose), pose_dir)
    
copy_files_dir(im_names, image_src, image_dir)
copy_files_dir(c_names, cloth_src, cloth_dir)
print("clothes and models you choose should be located in storage/data/test-end2end, please check (reload folder may take a few second)!!!")

clothes and models you choose should be located in storage/data/test-end2end, please check (reload folder may take a few second)!!!


### Data preprocessing all in one

In [42]:
cmd1 = "python segamentation/simple_extractor.py --dataset 'lip' --model-restore storage/checkpoints-humanParsing/exp-schp-201908261155-lip.pth --input-dir storage/data/test-end2end/image --output-dir storage/data/test-end2end/image-parse"
subprocess.call(cmd1, shell=True)

# add neck to segamentation
cmd2 = "python dataset_neck_skin_correction.py"
subprocess.call(cmd2, shell=True)

# binary masking
cmd3 = "python body_binary_masking.py"
subprocess.call(cmd3, shell=True)
# clothes mask
cmd4 = "python cloth_binary_masking.py"
subprocess.call(cmd4, shell=True)
#GMM
cmd5 = "python test.py --name GMM --dataroot storage/data --stage GMM --workers 4 --datamode test-end2end --data_list test_end2end_pairs.txt --checkpoint storage/checkpoints-cpvton-plus/GMM/gmm_opt.pth --result_dir storage/result-cpvton-plus-opt"
subprocess.call(cmd5, shell=True)
#Move GMM results to working directory
cmd6 = "mv -v storage/result-cpvton-plus-opt/GMM/test-end2end/warp-cloth storage/data/test-end2end"
subprocess.call(cmd6, shell=True)
cmd7 = "mv -v storage/result-cpvton-plus-opt/GMM/test-end2end/warp-mask storage/data/test-end2end"
subprocess.call(cmd7, shell=True)

0

### Now it's time to test cp-vton-plus-opt!

In [43]:
cmd8 = "python test.py --name TOM --dataroot storage/data --stage TOM --workers 4 --datamode test-end2end --data_list test_end2end_pairs.txt --checkpoint storage/checkpoints-cpvton-plus/TOM/tom_opt.pth --result_dir storage/result-cpvton-plus-testend2end"
subprocess.call(cmd8, shell=True)

0

### The results saved in storage/result-cpvton-plus-testend2end/TOM/test-end2end